# `comp-03`: Voxelwise encoding models
This demo introduces forward encoding models—that is, using regularized regression to predict voxelwise activity from an explicit model of the stimulus in left-out data. Here, we use a semantic forward encoding model capturing the meaning of words to predict fMRI activity while participants listen to a spoken narrative (similarly to e.g. [Huth et al., 2016](https://doi.org/10.1038/nature17637)). The words in each TR are assigned a vector (i.e. word embedding) representing their location in a high-dimensional semantic space. For each voxel, we'll fit a model that predicts the time series of brain activity from the semantic vectors assigned to eachtime point.

In [ ]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import pandas as pd
import json

### Naturalistic story-listening dataset
As an example, we'll use fMRI data collected for a single subject listening to a spoken story called "[I Knew You Were Black](https://themoth.org/stories/i-knew-you-were-black)" by Carol Daniel. These data are available as part of the publicly available [Narratives](https://github.com/snastase/narratives) collection ([Nastase et al., 2019](https://openneuro.org/datasets/ds002345)). Here, we'll download a single subject from the server for analysis. If you're working on the server, use `cp` on the command line to create a copy of the following file in your working directory; if you're working locally, use `scp` to download the file to your machine:

`/jukebox/PNI-classes/students/NEU502/2023-NEU502B/brainiak-aperture-data/sub-284_task-black_space-MNI152NLin2009cAsym_res-native_desc-clean_bold.nii.gz`

This dataset has been preprocessed using fMRIPrep with confound regression in AFNI. The functional data have been spatially normalized to a template in MNI space. To reduce computational demands, we compute parcel-wise ISCs using a cortical parcellation containing 400 parcels from Schaefer and colleages (2018). Load in the functional data and atlas. Use the parcellation to extract the mean functional time series for each parcel.

In [ ]:
# Filenames for intact notthefall data and Schaefer atlas
func_fn = ('sub-284_task-black_space-MNI152NLin2009cAsym_res-native_desc-clean_bold.nii.gz')
atlas_fn = ('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2.5mm.nii.gz')

# Load in the Schaefer 400-parcel atlas
atlas_nii = nib.load(atlas_fn)
atlas_img = atlas_nii.get_fdata()

# Load in intact functional data and compute parcel means:


To orient ourselves a bit, let's try plotting the BOLD time series for an example parcel. Start with parcel `196` in left superior temporal auditory association cortex. After plotting the time series, plot the location of this parcel in the brain using `plot_stat_map`.

In [ ]:
# Plot the time series for an example parcel:
from nilearn.plotting import plot_stat_map


# Plot parcel on MNI atlas:


### Semantic encoding model
We'll decompose the story stimulus into a series of semantic features based on a simple, widely used representation of word meaning called GloVe ([Pennington et al., 2014](http://dx.doi.org/10.3115/v1/D14-1162])). GloVe represents each word in a 300-dimensional semantic space where words that are nearer in space are more semantically similar based on co-occurrence statistics from a large corpus of text. We start with a time-stamped stimulus transcript. For each TR, we identify which words occurred in that TR, then assign that TR the semantic embeddings associated with each word. For TRs with multiple words, we average the semantic embeddings; for TRs with no words, we use a 300-dimensional vector of zeros; we exclude a small number of stop words. Finally, to account for the hemodynamic lag in fMRI, we create lagged versions of the embeddings.

In [ ]:
# Load in time-stamped transcript
transcript_fn = 'black_gentle.csv'
transcript = pd.read_csv(transcript_fn, sep=',')

# Stimulus is roughly 800 seconds long
tr = 1.5
stim_dur = 800
stim_trs = 800 // tr

# Convert transcript to list for simplicity
transcript = transcript.values.tolist()

# Loop through TRs
transcript_trs = []
for t in np.arange(stim_trs):
    
    # Container for words in this TR
    tr_words = []
    
    # Check if upcoming word onset is in this TR
    while t * tr < transcript[0][2] <= t * tr + tr:
        
        # If so, pop this word out of list and keep it
        w = transcript.pop(0)
        tr_words.append(w[0])
        
    # Append words and move to the next TR
    transcript_trs.append(tr_words)

In [ ]:
# Load in dictionary of GloVe embeddings
with open('glove_embeddings.json') as f:
    glove = json.load(f)
    
# Load list of standard stop words
stopwords = np.load('nltk_stopwords.npy')

Now that we have a transcript, a dictionary of word embeddings, and a list of stop words, we're ready to construct our predictor matrix. For each TR, obtain the words from `transcript_trs`. If a word is in our list of stop words, ignore it; otherwise, obtain the GloVe embedding for that word. If there are multiple words in a TR (excluding stop words), average their word embeddings to obtain a single embedding per TR. If a TR is empty (i.e. contains no words), set the embedding to a zero vector of the same dimensionality as the other embeddings. Lastly, visualize the resulting predictor matrix; z-score each column (i.e. GloVe dimension) for visualization.

In [ ]:
# Assign GloVe embeddings to each TR:


In [ ]:
# Visualize predictor matrix of GloVe embeddings:


Now, we'll horizontally stack the predictor matrix at lags of 2, 3, 4, and 5 TRs (3, 4.5, 6, 7.5 seconds) to account for variability in hemodynamic lag. Here's one recipe to do this: For each lag, first create prepending zero vectors for each TR in the lag; next, create appending zero vectors corresponding to the difference between the longest lag and the current lag; then, concatenate the prepending and appending vectors at the beginning and end of the predictor matrix, respectively. Horizontally stack the lagged matrices, z-score each column in the lagged predictor matrix, and visualize the matrix.

In [ ]:
# Create temporally delayed version of embeddings:
lags = [2, 3, 4, 5]

# Horizontally stack lagged embeddings:

# Z-score embeddings:
from scipy.stats import zscore

# Plot lagged predictor matrix:


The fMRI data were collected with an 8-TR buffer prior to the beginning of the story stimulus and a 9-TR buffer after the end of the story stimulus. Trim off the starting buffer TRs, as well as the ending buffer TRs (excluding TRs for the lags). This should finalize our predictor matrix.

In [ ]:
# Trim fMRI data to match embeddings:


### Ridge regression
Next, we use ridge regression to predict the activity at each parcel from the semantic embeddings. Note that the model dimensionality ($300 * 4$ delays $= 1200$ dimensions) is much greater than the number of samples ($<550$ TRs). This means we'll need to impose strong regularization on the model. Use a split-half outer cross-validation scheme where we train the model on half of the story and test the model on the other half. To search for the best-performing regularization parameter, perform 5-fold inner cross-validation within each training set using `RidgeCV`; this will select the best parameter setting from the inner cross-validation fold within the training half to predict the test half.

In [ ]:
# Split-half outer cross-validation fold:
from sklearn.model_selection import KFold


# Ridge regression with alpha grid and nested CV:
from sklearn.linear_model import RidgeCV
alphas = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]

# Loop through outer split-half cross-validation:


### Evaluating model performance
To evaluate the predictions of our model, we use the model weights estimated from the training data to predict the brain activity from semantic embeddings for the test data. We then assess the similarity between the predicted brain activity and the actual brain activity for the test data. Keeping with conventions in the literature, we use Pearson correlation to assess the match between predicted and actual brain activity (although this is not considered a good scoring metric in machine learning and there are many other options; e.g. $R^2$). For each parcel, compute the Pearson correlation between the actual and predicted test time series. Plot the predicted and actual time series for the superior temporal parcel `196` (z-score the actual and predicted time series for plotting).

In [ ]:
# Compute correlation between predicted and actual responses:
from scipy.stats import pearsonr

    
# Plot predicted and actual response for example parcel:


Finally, we'll plot the prediction performance scores on the brain. Convert the parcel-level prediction scores back into voxel-level brain maps; that is, for every voxel in a given parcel, assign that voxel the parcel's prediction score. Visualize the resulting brain map using `plot_stat_map`.

In [ ]:
# Plot correlations between actual and predicted responses:
from nilearn.plotting import plot_stat_map



#### References

* Huth, A. G., De Heer, W. A., Griffiths, T. L., Theunissen, F. E., & Gallant, J. L. (2016). Natural speech reveals the semantic maps that tile human cerebral cortex. *Nature*, *532*(7600), 453-458. https://doi.org/10.1038/nature17637

* Nastase, S. A., Liu, Y.-F., Hillman, H., Zadbood, A., Hasenfratz, L., Keshavarzian, N., Chen, J., Honey, C. J., Yeshurun, Y., Regev, M., Nguyen, M., Chang, C. H. C., Baldassano, C., Lositsky, O., Simony, E., Chow, M. A., Leong, Y. C., Brooks, P. P., Micciche, E., Choe, G., Goldstein, A., Vanderwal, T., Halchenko, Y. O., Norman, K. A., & Hasson, U. (2020). Narratives: fMRI data for evaluating models of naturalistic language comprehension. *bioRxiv*. https://doi.org/10.1101/2020.12.23.424091

* Pennington, J., Socher, R., & Manning, C. D. (2014, October). GloVe: Global Vectors for Word Representation. In *Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP)* (pp. 1532-1543). https://www.aclweb.org/anthology/D14-1162

* Schaefer, A., Kong, R., Gordon, E. M., Laumann, T. O., Zuo, X. N., Holmes, A. J., ... & Yeo, B. T. (2018). Local-global parcellation of the human cerebral cortex from intrinsic functional connectivity MRI. Cerebral cortex, 28(9), 3095-3114. https://doi.org/10.1093/cercor/bhx179